In [1]:
# Imports
import gradio as gr
import g4f
import tiktoken

In [10]:
# Start settings
system_message = [{"role": "system", "content": "Ты пират, отвечай как пират."}]
max_response_tokens = 1024
token_limit = 16000
last_message = "" 
conversation = system_message.copy()

c:\Users\KorolOrol\AppData\Local\Programs\Python\Python310\lib\site-packages\gradio\components\button.py:89: UserWarning: Using the update method is deprecated. Simply return a new object instead, e.g. `return gr.Button(...)` instead of `return gr.Button.update(...)`.
  warnings.warn(
Traceback (most recent call last):
  File "c:\Users\KorolOrol\AppData\Local\Programs\Python\Python310\lib\site-packages\gradio\queueing.py", line 406, in call_prediction
    output = await route_utils.call_process_api(
  File "c:\Users\KorolOrol\AppData\Local\Programs\Python\Python310\lib\site-packages\gradio\route_utils.py", line 226, in call_process_api
    output = await app.get_blocks().process_api(
  File "c:\Users\KorolOrol\AppData\Local\Programs\Python\Python310\lib\site-packages\gradio\blocks.py", line 1563, in process_api
    data = self.postprocess_data(fn_index, result["prediction"], state)
  File "c:\Users\KorolOrol\AppData\Local\Programs\Python\Python310\lib\site-packages\gradio\blocks.py", l

In [3]:
# Count tokens
def num_tokens_from_messages(messages):
    encoding= tiktoken.get_encoding("cl100k_base")  #model to encoding mapping https://github.com/openai/tiktoken/blob/main/tiktoken/model.py
    num_tokens = 0
    for message in messages:
        num_tokens += 4  # every message follows <im_start>{role/name}\n{content}<im_end>\n
        for key, value in message.items():
            num_tokens += len(encoding.encode(value))
            if key == "name":  # if there's a name, the role is omitted
                num_tokens += -1  # role is always required and always 1 token
    num_tokens += 2  # every reply is primed with <im_start>assistant
    return num_tokens

In [4]:
# Convert history to openai format
def gradio_to_openai_history(history):
    result = []
    for dialogue in history:
        result.append({"role": "user", "content": dialogue[0]})
        result.append({"role": "assistant", "content": dialogue[1]})
    return result

In [14]:
# Chat response
def response(message, history):
    conversation = system_message + gradio_to_openai_history(history) + [{"role": "user", "content": message}]
    conv_history_tokens = num_tokens_from_messages(conversation)

    while conv_history_tokens + max_response_tokens >= token_limit:
        del conversation[1] 
        conv_history_tokens = num_tokens_from_messages(conversation)
    
    answer = g4f.ChatCompletion.create(
        model="gpt-3.5-turbo-16k-0613",
        provider=g4f.Provider.NeuroGPT,
        messages=conversation,
        stream=True,
        )
    full_message = ""
    for message in answer:
        full_message += message
        yield full_message

In [6]:
def find_item():
    return g4f.ChatCompletion.create(
        model="gpt-3.5-turbo-16k-0613",
        provider=g4f.Provider.NeuroGPT,
        messages=[{"role": "system", "content": "Проанализируй данный текст и найди в нем предмет. Если здесь нет предмета, то напиши 'нет'."},
                  {"role": "user", "content": last_message}],
        stream=False,
        )

In [16]:
with gr.Blocks() as demo:
    chatbot = gr.ChatInterface(response).queue()
    find = gr.Button("Find")
    find.click(find_item, outputs=gr.Text(), show_progress=True, status_tracker=None)
demo.queue().launch()

Running on local URL:  http://127.0.0.1:7864

To create a public link, set `share=True` in `launch()`.


c:\Users\KorolOrol\AppData\Local\Programs\Python\Python310\lib\site-packages\gradio\components\button.py:89: UserWarning: Using the update method is deprecated. Simply return a new object instead, e.g. `return gr.Button(...)` instead of `return gr.Button.update(...)`.
  warnings.warn(
Traceback (most recent call last):
  File "c:\Users\KorolOrol\AppData\Local\Programs\Python\Python310\lib\site-packages\urllib3\response.py", line 710, in _error_catcher
    yield
  File "c:\Users\KorolOrol\AppData\Local\Programs\Python\Python310\lib\site-packages\urllib3\response.py", line 1073, in read_chunked
    self._update_chunk_length()
  File "c:\Users\KorolOrol\AppData\Local\Programs\Python\Python310\lib\site-packages\urllib3\response.py", line 1008, in _update_chunk_length
    raise InvalidChunkLength(self, line) from None
urllib3.exceptions.InvalidChunkLength: InvalidChunkLength(got length b'', 0 bytes read)

The above exception was the direct cause of the following exception:

Traceback (most 

In [ ]:
print(last_message)